### Import the Data

In [20]:
import numpy as np
from sklearn import preprocessing

In [21]:
#Load the data
raw_csv_data = np.loadtxt('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data.csv',delimiter=',')

In [22]:
# The inputs are all columns in the csv, except for the first one [:,0]
# (which is just the arbitrary customer IDs that bear no useful information),
# and the last one [:,-1] (which is the targets)
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [23]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

In [24]:
# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# Create a "balanced" dataset, remove(should be) some input/target pairs.
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# Delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [25]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [26]:
# Data should be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [27]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1792.0 3579 0.5006985191394244
214.0 447 0.47874720357941836
231.0 448 0.515625


### Save the three datasets in *.npz

In [28]:
np.savez('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Import Data

In [29]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

In [30]:
# Create a temporary variable npz, where each of the three Audiobooks datasets will be stored
npz = np.load('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data_train.npz')

# Extract the inputs using the keyword under which they saved 
# to ensure that they are all floats
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (to one-hot encode)
train_targets = npz['targets'].astype(np.int)

/var/folders/q_/w5j7802d5gl97srncmkrtn3w0000gn/T/ipykernel_1575/2895031299.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
/var/folders/q_/w5j7802d5gl97srncmkrtn3w0000gn/T/ipykernel_1575/2895031299.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and

In [31]:
# Load the validation data in the temporary variable
npz = np.load('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data_validation.npz')

# Load the inputs and the targets
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

/var/folders/q_/w5j7802d5gl97srncmkrtn3w0000gn/T/ipykernel_1575/4059420204.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
/var/folders/q_/w5j7802d5gl97srncmkrtn3w0000gn/T/ipykernel_1575/4059420204.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional inf

In [32]:
# Load the test data in the temporary variable
npz = np.load('/Users/kyotun/Desktop/ML/audiobook/data/Audiobooks_data_test.npz')

# Create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

/var/folders/q_/w5j7802d5gl97srncmkrtn3w0000gn/T/ipykernel_1575/934515054.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
/var/folders/q_/w5j7802d5gl97srncmkrtn3w0000gn/T/ipykernel_1575/934515054.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depr

### Model

In [33]:
# Set the input and output sizes
input_size = 10
output_size = 2

# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50

In [34]:
# Define how the model will look like
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choose the optimizer and the loss function

In [35]:
# Define the optimizer,
# the loss function, 
# and the metrics to obtain at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [36]:
# Set the batch size
batch_size = 100

# Set a maximum number of training epochs
max_epochs = 100

In [37]:
# Set an early stopping mechanism
# patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

In [38]:
# Fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs to train(assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure to get enough information about the training process
          )  

Epoch 1/100


36/36 - 0s - loss: 0.6213 - accuracy: 0.6449 - val_loss: 0.5174 - val_accuracy: 0.7494 - 374ms/epoch - 10ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4836 - accuracy: 0.7578 - val_loss: 0.4263 - val_accuracy: 0.7830 - 54ms/epoch - 1ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4295 - accuracy: 0.7754 - val_loss: 0.3882 - val_accuracy: 0.7875 - 51ms/epoch - 1ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4004 - accuracy: 0.7768 - val_loss: 0.3625 - val_accuracy: 0.8098 - 46ms/epoch - 1ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3850 - accuracy: 0.7907 - val_loss: 0.3556 - val_accuracy: 0.8009 - 40ms/epoch - 1ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3764 - accuracy: 0.7952 - val_loss: 0.3412 - val_accuracy: 0.8076 - 37ms/epoch - 1ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3707 - accuracy: 0.8044 - val_loss: 0.3399 - val_accuracy: 0.8031 - 33ms/epoch - 923us/step
Epoch 8/100
36/36 - 0s - loss: 0.3665 - accuracy: 0.8019 - val_loss: 0.3332 - val_accuracy: 0.8076 - 34ms/epoch - 939us/step
Epoch 9/100
36/36 - 0s

### Test the Model

In [39]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 553us/step - loss: 0.2947 - accuracy: 0.8549


In [40]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.29. Test accuracy: 85.49%
